In [112]:
import json

article_urls = json.load(open('test-data/article-urls.json'))
promise_titles = json.load(open('test-data/promise-titles.json'))

promise_texts = []
for idx, title in enumerate(promise_titles):
    promise_texts.append(open('../promisedata/korean/prom%d.txt' % (idx+1)).read())

In [115]:
import requests
import re
def download_article(url):
    article_id = re.search('aid=(\d+)', url).group(1)
    fname = 'test-data/article-%s.json' % article_id
    api_url = 'https://api.budgetwiser.org/api/news/get_by_url/'
    article_parsed = requests.get(api_url, {'url': url}).json()
    with open(fname, 'w') as f:
        json.dump(article_parsed, f)

import os.path
def load_article(url):
    article_id = re.search('aid=(\d+)', url).group(1)
    fname = 'test-data/article-%s.json' % article_id
    if not os.path.isfile(fname):
        download_article(url)
    return json.load(open(fname))

In [288]:
import logging
from konlpy.tag import Twitter, Kkma
twitter = Twitter()
kkma = Kkma()
import string
import re
remove_punct_map = dict.fromkeys(map(ord, string.punctuation))
remove_quotes_map = dict([(ord(x), " ") for x in ".․,‘’´“”·‧<>「」【】–-()~～…"]) 
def clean_title(title):
    # Remove punctuation and quotes
    title = title.translate(remove_punct_map)
    title = title.translate(remove_quotes_map)
    # Collapse double spaces
    title = re.sub('\s+', ' ', title)
    return title

stopwords = "수 년 등 및 몇 중 네이버 뉴스".split()
stopwords += "공급 설치 조성 운영 실행 설립 확대 제공 실시 지원 검토 육성 추진 유치 강화 개선 구축 마련 확충 실시 개선 해소".split()
stopwords = set(stopwords)
pos = lambda d: ['/'.join(p) for p in twitter.pos(d, stem=True, norm=True) if p[0] not in stopwords and p[1] in ('Noun', 'Verb', 'Alpha', 'Adjective')]
#pos = lambda d: [p for p in kkma.morphs(clean_title(d)) if p not in stopwords]

In [292]:
# Load LSI model and index
from gensim import similarities, models, corpora

dictionary_ko = corpora.Dictionary.load('ko.dict')
index = similarities.MatrixSimilarity.load('lsi.index')
lsi = models.LsiModel.load('lsi.model')

def find_similar(tagged_text, similarity_threshold=None):
    vec_bow = dictionary_ko.doc2bow(tagged_text)
    vec_lsi = lsi[vec_bow]
    sims = enumerate(index[vec_lsi])
    if similarity_threshold:
        sims = filter(lambda item: item[1] >= similarity_threshold, sims)
    return sorted(sims, key=lambda item: -item[1])


In [293]:
# Load manually tagged article-promise matches
import csv

article_promise_manual = {}
with open('test-data/article-promise-manual.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        article_title = row[0]
        _promise_titles = [t for t in row[1:] if t]
        if article_title:
            article_promise_manual[article_title] = _promise_titles

In [298]:
%%time
import numpy as np

print("thresh\tprec\trec\tf1")

verbose = True

#for similarity_threshold in np.arange(0.7, 0.9, 0.02):
for similarity_threshold in (0.75,):
    k_precision_values = []
    k_recall_values = []
    all_ranks = []

    for url in article_urls:
        article_parsed = load_article(url)
        # title + excerpt
        n_words = 100
        text_excerpt = " ".join(list(filter(lambda word: len(word)>1, clean_title(article_parsed['text']).replace("\n", " ").split(" ")))[:n_words])
        search_text = article_parsed['title'] + " " + text_excerpt
        similar_promises = find_similar(pos(search_text))
        if verbose:
            print("Article: %s" % article_parsed['title'])
            print(text_excerpt)

        manual_promises = article_promise_manual[article_parsed['title']]

        num_relevant = len(manual_promises)
        recommended = list(filter(lambda item: item[1] >= similarity_threshold, similar_promises))
        num_recommended = len(recommended)

        if verbose:
            print("  Top-5 (of %d) algorithm matched promises:" % num_recommended)
            for rank, item in enumerate(recommended[:5]):
                print("  (Rank %d, %d%%) %s" % (rank, item[1]*100, promise_titles[item[0]]))

            print("  Manually tagged promises:")
            if not manual_promises:
                print("  (None)")

        ranks = []    
        for item in manual_promises:
            promise_idx = promise_titles.index(item)
            rank_score = (0, 0)
            for rank, sim in enumerate(similar_promises):
                if sim[0] == promise_idx:
                    rank_score = (rank, sim[1])
                    break
            if verbose:
                print("  (Rank %d, %d%%) %s" % (rank_score[0], rank_score[1]*100, item))
            ranks.append(rank_score[0])


        if num_recommended > 0:
            max_rank = num_recommended - 1
            # Count the number of correctly recommended items by comparing ranks from prediction and ground truth
            num_relevant_recommended = len(list(filter(lambda r: r <= max_rank, ranks)))
            precision_at_k = num_relevant_recommended / num_recommended
            if num_relevant > 0:
                recall_at_k = num_relevant_recommended / num_relevant
            else:
                recall_at_k = 1  # nothing relevant, so always right
        else:
            precision_at_k = 1  # nothing recommended, so nothing wrong
            recall_at_k = 0  # nothing recommended, so nothing right

        k_precision_values.append(precision_at_k)
        k_recall_values.append(recall_at_k)
        all_ranks += ranks

        if verbose:
            print("Precision@%d = %.4f   Recall@%d = %.4f" % (num_recommended, precision_at_k, num_relevant, recall_at_k))
            print()

    prec = sum(k_precision_values)/len(k_precision_values)
    rec = sum(k_recall_values)/len(k_recall_values)
    f1 = 2*(prec*rec)/(prec+rec)
    print("%.3f\t%.4f\t%.4f\t%.4f" % (similarity_threshold, prec, rec, f1))
#print("Average precision: %.4f" % (prec))
#print("Average recall: %.4f" % (rec))
#print("F1 score: %.4f" % (f1))
#print("Average rank: %.1f" % (sum(all_ranks)/len(all_ranks)))


thresh	prec	rec	f1
Article: 국세청, 고액·상습 체납자 2만1403명 명단 공개...김우중 '369억원' 체납
서울뉴시스 서울뉴시스 서울뉴시스 서울뉴시스 고액 체납자 1위에 유지양 효자건설 회장447억원 체납유병언 세모그룹 회장 자녀 증여서 115억원 납부안해재산 추적조사 통해 1조5752억원 세금 징수193명 형사고발 세종뉴시스 박상영 기자 국세청은 11일 2억원 이상 세금을 내지 않은 고액 상습 체납자 2만1403명의 명단을 공개했다국세청은 국세 체납 이후 1년 넘게 내지 않고 버티고 있는 개인이나 법인의 명단을 매년 공개하고 있다 올해는 기준금액이 체납 3억원에서 2억원 이상으로 낮아지면서 공개인원이 작년보다 4748명 늘어났다이번에 공개된 2만1403명 개인은 1만5027명 법인은 6376곳 이었다 체납액은 11조34697억원으로 전년13조3018억원보다 8321억원 감소했다체납액 규모는 2억 5억원 구간의 인원이 1만6931명으로 전체의 792 체납액은 6조7977억원으로 전체의 593를 차지했다올해 공개된 고액 상습 체납자 1위는 유지양 효자건설 회장으로 상속세 447억원을
  Top-5 (of 2) algorithm matched promises:
  (Rank 0, 83%) 시민 모니터링요원 및 전담조직 신설
  (Rank 1, 83%) 불공정피해 상담센터 기능 확대
  Manually tagged promises:
  (None)
Precision@2 = 0.0000   Recall@0 = 1.0000

Article: 공공기관 채용비리 '칼바람'…옷 벗는 기관장 속출할 듯
김용진 기획재정부 2차관이 8일 서울 세종로 정부서울청사에서 합동브리핑을 열고 공공기관 채용비리 전수조사 점검 결과을 발표하고 있다 김용진 2차관은 공공기관 채용비리 관련 전수조사 결과 2234건이 적발됐다며 앞으로도 공공기관 채용비리를 상시적으로 점검하겠다고 밝혔다 왼쪽부터 안준호 국민권익위원회 부패방지국장 허경렬 경찰청 수사국장 2차관 조규홍 기획재

Article: "학부모 감당 힘들어…" 초등교사들 江南근무 손사래
도를 넘는 민원을 겪고 나니 하루라도 빨리 강남을 벗어나고 싶더라고요 교장 교감도 힘센 학부모 항의에 한마디도 하고 교사에게 무조건 죄송하다고 하라고 합니다서울 서초구의 초등학교에 근무하는 이모41 교사는 올해만 넘기면 강남 서초 학교에서 다시는 근무하지 않겠다고 말했다 지난해 2학년 학생들끼리 말다툼을 벌여 타이르고 화해시켰는데 한쪽 부모가 학교폭력위를 열라고 요구하더니 아이 큰아빠가 판사라며 소송을 걸겠다고 찾아오는 동안 시달렸다는 것이다 교사는 출퇴근이 조금 힘들어지더라도 학생 지도가 쉬운 동네로 학교를 옮기고 싶다고 말했다최근 서울에서 초등 교사 강남 공동화空洞化 현상이 벌어지고 있다 학부모 민원 등의 이유로 강남 서초구 초등학교 근무를 기피하는 중견 교사가 늘고 있기 때문이다 이들이 빠져나간 자리는 초임 교사가 메우고 있다 서울시교육청 관계자는
  Top-5 (of 8) algorithm matched promises:
  (Rank 0, 83%) 어린이 안전을 위한 초등학교 스쿨버스 도입
  (Rank 1, 79%) 시민카드로 마음껏 빌려가세요
  (Rank 2, 78%) 학교폭력 대처 방법, 서울시가 알려드립니다
  (Rank 3, 78%) 어린이 체험학습 프로그램, 일목요연하게 모아드려요
  (Rank 4, 78%) 노인상대 사기범죄! ‘실버 보안관’이 지킨다
  Manually tagged promises:
  (None)
Precision@8 = 0.0000   Recall@0 = 1.0000

Article: 신촌 대학가에 월세 10만원대 공공기숙사 생긴다
광흥창역 역세권 청년주택 조감도서울시 제공 광흥창역 역세권 청년주택 조감도서울시 제공 광흥창역 역세권 청년주택 활용서울연합뉴스 박초롱 기자 연세대 이화여대 서강대 홍익대 대학들이 밀집한 신촌 역세권에 10만∼12만원에 입주할 있는 공공기숙사가 생긴다서울시는 마포구 광흥창역 인근에 짓는 역세권 청년주택을 활용해 공공기숙사를

Article: “육아 해결할 길 없어 일 포기” 82년생 김지영들의 좌절
통계 증언으로 82년생 김지영들 이미지를 클릭하면 확대됩니다 이미지를 클릭하면 확대됩니다 이미지를 클릭하면 확대됩니다 이미지를 클릭하면 확대됩니다 82년생 김지영민음사 2016 30대 여성이 경험하는 일상적 차별과 구조적 불평등을 사실적으로 그려낸 조남주 작가의 소설 슬하에 딸을 서른네살 전업주부 김지영씨는 어느 갑자기 친정어머니 등으로 빙의하는 증세를 보인다 남편이 주선한 정신 상담에서 김지영씨는 자신의 삶을 이야기한다 소설은 김지영씨의 이야기를 들은 담당 의사가 그의 인생을 재구성해 기록한 리포트 형식이다 지난해 10월 출간 지금까지 38만부 넘게 판매됐다 Weconomy 홈페이지 바로가기 httpwwwhanicokrartieconomy Weconomy 페이스북 바로가기 httpswwwfacebookcomeconohani 한겨레출판사 디자이너로 12년간 일해온 이수경가명 35씨는 지난달 회사를 그만두고 남편 직장이 있는 제주도로 이사 갔다 그는 3년 결혼한 주말부부로 살며 첫아이를 낳았다
  Top-5 (of 0) algorithm matched promises:
  Manually tagged promises:
  (Rank 55, 45%) 국공립어린이집 1,000개소 확충
  (Rank 134, 31%) 0~12세 아동에 대한 주치의제도 실시
  (Rank 8, 59%) 맞춤형 여성일자리 10만개 창출
  (Rank 183, 19%) 우리 아이 돌보미, 서울시가 찾아드려요
  (Rank 147, 28%) ‘어린이 집 아동보호수칙’이 있잖아요
  (Rank 33, 49%) 공공서비스 중심 맞춤형 100대 적합업종 발굴
Precision@0 = 1.0000   Recall@6 = 0.0000

Article: [뒤끝뉴스] 빚 잘 갚는 사람만 손해 본다?
정부가 나서서 연체를 부추기는 정책이다 이자 내는 사람만 오히려 손해를 본다 정부가 조만간 내놓을 탕감 정책이나 연체자 지원 방안을

  (Rank 109, 35%) <상가임차인 보호를 위한 특별조례> 제정, 정기적인 실태조사 및 상가임대차 상담.분쟁조정 확대 실시
  (Rank 127, 31%) ‘서울형 장기 안심상가’시범 도입
  (Rank 6, 76%) 위대한 골목가게 탄생, ‘우리 가게 콘테스트’
  (Rank 8, 75%) 동네 위해 헌신하신 사장님의 공헌을 인증합니다
Precision@10 = 0.3000   Recall@6 = 0.5000

Article: 의무휴업제 시행 5년···대형마트·전통시장 둘다 침체된 까닭은?
서울뉴시스 최선윤 기자 대형마트 의무휴업제가 시행된 5년이 지난 가운데 당초 규제 취지와 달리 대형마트 전통시장 모두의 매출액이 감소하는 부정적인 효과가 나타나고 있다는 지적이 제기되면서 까닭은 무엇일 관심이 쏠린다27일 관련업계에 따르면 대형마트 의무휴업제가 도입된 배경은 전통시장 소상공인 보호에 있다 대형마트가 급격히 성장하자 해당 지역에서 상권을 공유하고 있는 중소유통업체와 소상공인들이 대형유통업체의 사업 확장에 대한 규제를 도입해야 한다는 주장을 지속했다이에 따라 지난 2005년부터 영업시간 영업품목 제한 대형유통업체의 사업 확장 규제를 내용으로 하는 다수의 의원입법안이 국회에 제출됐고 2012년 1월17일 유통산업발전법 일부개정법률안이 공포됨에 따라 지방자치단체는 조례를 통해 대규모점포 대형마트와 준대규모점포의 영업을 제한할 있게 됐다대형유통업체들로 구성된 한국체인스토어협회는 영업시간 제한과 의무 휴업일 강제가 소비자 불편과 소비 위축을
  Top-5 (of 14) algorithm matched promises:
  (Rank 0, 98%) 위대한 골목가게 탄생, ‘우리 가게 콘테스트’
  (Rank 1, 98%) 자영업 지원센터’ 설치
  (Rank 2, 95%) 동네 위해 헌신하신 사장님의 공헌을 인증합니다
  (Rank 3, 94%) 골목경제 협동조합을 위한 공동협업사업 지원
  (Rank 4, 90%) 전통시장 장보기, 서울시가 보관해 드립니다


  (Rank 4, 77%) 시민 큐레이터에 도전해 보세요
  Manually tagged promises:
  (None)
Precision@6 = 0.0000   Recall@0 = 1.0000

Article: "한국 스마트폰 데이터요금 세계 주요 41개국 중 가장 비싸"
한국 스마트폰 데이터요금 세계 주요국 가장 비싸서울연합뉴스 신준희 기자 우리나라의 스마트폰 데이터 요금이 세계 주요 나라 가운데 가장 비싸다는 분석이 나왔다 핀란드의 국제 경영컨설팅 업체인 리휠이 경제개발협력기구OECD와 유럽연합EU에 속한 41개국 187개 이동통신업체재판매업체 58개 포함의 요금제 1천628개를 비교 분석한 결과 이같이 나타났다 사진은 5일 서울시내 휴대전화 판매점 hamaynacokr 한국 스마트폰 데이터요금 세계 주요국 가장 비싸서울연합뉴스 신준희 기자 우리나라의 스마트폰 데이터 요금이 세계 주요 나라 가운데 가장 비싸다는 분석이 나왔다 핀란드의 국제 경영컨설팅 업체인 리휠이 경제개발협력기구OECD와 유럽연합EU에 속한 41개국 187개 이동통신업체재판매업체 58개 포함의 요금제 1천628개를 비교 분석한 결과 이같이 나타났다사진은 5일 서울시내 휴대전화 판매점 hamaynacokr 통신 3사 데이터요금제는 담합 서울연합뉴스 권준우 기자
  Top-5 (of 2) algorithm matched promises:
  (Rank 0, 83%) 한양도성 세계유산 등재
  (Rank 1, 83%) 도심 광폭도로에 대한 도로 다이어트 추진
  Manually tagged promises:
  (Rank 16, 42%) 서울전역 와이파이존 구축
Precision@2 = 0.0000   Recall@1 = 0.0000

Article: 유아까지 코딩 열풍…SW수업 의무화에 사교육시장 '꿈틀'
세종연합뉴스 고유선 기자 내년부터 학교에서 소프트웨어SW 교육이 필수화하면서 코딩컴퓨터 프로그래밍 교육과 관련된 사교육 시장이 들썩이고 있다5일 교육계에 따르면 대치동 반포동 목동을 비